# Homework 2 (Problem 5, 2pts)
## ME 334: Advanced Dynamics, Control and System Identification
### Stanford University

## Problem Statement (Non-Holonomic Robot Control)


![nonholonomicrobot_2.jpg](attachment:nonholonomicrobot_2.jpg)



### Kinematics

Here we have a non-holonomic (meaning its linear velocity is restricted so that the forward velocity points along $\hat{r}_{1}$). The position of reference frame $R$ is $^A{r}^R=x_R \hat{
a}_1 + y_R \hat{a}_2$, its velocity $^A{v}^R = \dot{x}_R \hat{a}_1 + \dot{y}_R \hat{a}_2$ and must respect the non-integrable, knife-edge non-holonomic constraint meaning $^R{v}=v\hat{r}_1 + 0 \hat{r}_2 + 0\hat{r}_3$ and for a constraint vector $^A\lambda = -sin(\theta)\hat{a}_1 + cos(\theta)\hat{a}_2$, the nonholonomic constraint is written as:

$$-sin(\theta)\dot{x}_R + cos(\theta) \dot{y}_R = 0$$

For the rotation from frame $R$ to frame $A$, the rotation is about the axis $\hat{a}_1$ through angle $\theta$:

$$ \begin{bmatrix}\hat{a}_1 \\ \hat{a}_2 \\ \hat{a}_3   \end{bmatrix} = \underbrace{\begin{bmatrix} cos(\theta) & -sin(\theta) & 0 \\ sin(\theta) & cos(\theta) & 0 \\ 0 & 0 & 1 \end{bmatrix}}_{^AR_R} \begin{bmatrix} \hat{r}_1 \\ \hat{r}_2 \\ \hat{r}_3 \end{bmatrix} $$

Due to the holonomic constraint, if a desired point $^A{r}^D(t)$ were to lie on the non-holonomic constraint vector $^A\lambda$ then the immediate velocity of the robot could not move in that direction due to the constraint. One strategy would be to rotate in-place first until the $\hat{r}_{1}$ axis were aligned with the target position. While this strategy would work well if the goal were static, if we take the case where the goal is on a circle with the robot at the center as was moving on the circle such that it stayed on the non-holonomic line as the robot rotated then the robot would simply rotate in place at the center of the circle.

But if we consider this scenario, we intuitively know that the robot could apply a small forward velocity and essentially meet the desired point on the circle by following the path of the spiral until it reached the circle containing the desired point.

*To achieve this, we must consider the control of a point that does not sit on the non-holonomic constraint line (as $R$ frame sits on the non-holonomic constraint line)*. Let's consider a point P (shown in the figure, fixed in frame $R$) that sits at $^Rr^P=L \hat{r}_1 + 0 \hat{r}_2 + 0 \hat{r}_3$. This point in frame A is:

$$^Ar^P = ^Ar^R + [^AR_R] ^Rr^P$$
Differentiating we have:
$$^A{\dot{r}}^P = ^A{\dot{r}}^R + [^A{\dot{R}}_R] ^Rr^P + [^AR_R] ^R{\dot{r}}^P$$
we know by definition that $^R{\dot{r}}^P=0$, and

$$[^A{\dot{R}}_R] = [^A \omega ^R]^{[\times]}[^AR_R]$$

where $^A\omega ^R = 0\hat{a}_1 + 0 \hat{a}_2 + \dot{\theta} \hat{a}_3$. We also have that  $^A{\dot{r}}^R = [^A R_R] ^R{v}$:
$$^A{\dot{r}}^R = ^A R_R \begin{bmatrix} v\hat{r}_1 \\ 0 \hat{r}_2 \\ 0\hat{r}_3  \end{bmatrix}  $$

Substituting these above we get a solution of the form
$$^A{\dot{r}}^P = [^A R_R] ^R{v} + [^A \omega ^R]^{[\times]}[^AR_R] ^Rr^P $$
If we simplify this to only include the xy-position in the A frame (remove the z-component) we can simplify to:
$$\begin{bmatrix}
^A{\dot{r}}^P_x \\ ^A{\dot{r}}^P_y
\end{bmatrix} = M_{2\times 2} \begin{bmatrix}
v \\ \dot{\theta}
\end{bmatrix} $$

This is the matrix you will show is full rank (non-zero determinant), and use for control of the xy-position in the next part:

#### Functions
1. you will make the function that returns the xy-position of point P: $^A{{r}}^P_{xy}$ in a $2\times1$ numpy matrix (e.g. ```np.matrix([[2][4]]) = np.matrix([2,4]).T```) given $^A{{r}}^R_{xy}$, $L$ and $\theta$:
$$^A{{r}}^P_{xy} = f_1(^A{{r}}^R_{xy}, L,\theta) $$
2. You will make a function that calculate the above matrix $M$ ($2\times 2$) given $L$, $\theta$:
$$ M_{2\times 2} = f_2(L,\theta)$$

2D numpy matrix: ```np.matrix([[1,2],[3,4]]) ``` $=\begin{bmatrix}1 & 2 \\ 3 & 4 \end{bmatrix}$ (if you need to get the inverse at some point later: $M^{-1}$: ```M_inv = np.linalg.inv(M)```)

### Control

For control, you will be given a 2D xy-position point to track $^Ar^D_{xy}(t) = x_d(t) \hat{a}_1 + y_d(t) \hat{a}_2 $, the input to this function will also be the current state of the robot $^A{{r}}^R_{xy}(t)$ and angle $\theta$. In this function you will want to
- Obtain the point P: $^A{{r}}^P_{xy}(t)$ leveraging your earlier function and $^A{{r}}^R_{xy}(t)$, $L$, $\theta$
- Find the xy-error: $^A e(t) = ^Ar^D_{xy}(t) - ^A{{r}}^P_{xy}(t)$
- Obtain the current matrix $M$: $ M_{2\times 2}(t) = f_2(L,\theta(T))$
- Calculate your control law, I suggest something along the lines of
$$u_{2\times 1}(t)= \begin{bmatrix}v \\ \dot{\theta} \end{bmatrix}_{2\times 1}= M^{(-1)}_{2\times 2}[K_p]_{2\times 2} [^A e_{xy}]_{2\times 1}(t)$$
where $[K_p]_{2\times 2} = \begin{bmatrix} K & 0 \\ 0 & K \end{bmatrix}$ for $K>0$.

#### Function
1. You will write a controller that provides a control law:
$$ u_{2\times 1}(t) = f_3([^Ar^D]_{xy}(t), [^A{{r}}^R]_{xy}(t), \theta(t), L)$$
with the output being a $2\times 1$ numpy matrix.

## Alternate Expression for Kinematics
You can show the the velocity of the robot frame can be represented in the following way:
$$\begin{bmatrix}^A\dot{x}_R \\ ^A\dot{y}_R \\ \dot{\theta}  \end{bmatrix} = \begin{bmatrix} cos(\theta) & 0 \\
sin(\theta) & 0 \\
0 & 1\end{bmatrix} \underbrace{\begin{bmatrix} v \\ \dot{\theta} \end{bmatrix}}_{u(t)} $$


## Top-Level

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import numpy as np
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp
%matplotlib notebook

from matplotlib import rc
rc('animation', html='jshtml')

In [ ]:
# parameters
max_time = 30
dt = 1. / 20.
t = np.linspace(0, max_time, int(max_time / dt) + 1)
L = 0.01

## Your Code

In [ ]:
def student_function_1(r, theta, L):
    """Returns the xy position of point P in frame A

    Parameters
    ----------
    r : np.ndarray, shape=(2,)
        Current position of frame R with respect to frame A.
    theta : float
        Current rotation angle about axis a_3.
    L : float
        Position of P in frame R along axis r_1.

    Returns
    -------
    p : np.ndarray, shape=(2,)
        Position of P in frame A.
    """
    # Rotation matrix from frame R to frame A.
    R = np.array([[np.cos(theta), -np.sin(theta), 0],
                  [np.sin(theta), np.cos(theta), 0],
                  [0, 0, 1]
                  ])
    p = r + R @ np.array([L, 0, 0])
    return p

def student_function_2(L, theta):
    """Returns the matrix M.

    Parameters
    ----------
    See student_function_1.

    Returns
    -------
    M : np.ndarray, shape=(2, 2)
        The matrix M relating input velocities to the velocity of point P.
    """
    M = np.array([[np.cos(theta), -L * np.sin(theta)],
                  [np.sin(theta), L * np.cos(theta)]])
    return M

def student_function_3(rd, r, L, theta):
    """Computes the control input u=[v, omega]^T.

    Parameters
    ----------
    rd : np.ndarray, shape=(2,)
        Desired position of robot position.
    See student_function_1 for rest of parameters.

    Returns
    -------
    u : np.ndarray, shape=(2,)
        Control input [v, omega]^T.
    """
    M = student_function_2(L, theta)
    u = np.linalg.inv(M) @ (r - rd)
    return u

## Simulation

In [ ]:
class Simulation(object):
    def __init__(self, rd_type='circle'):
        assert rd_type == 'circle' or rd_type == 'sine'
        self.rd_type = rd_type

    def desired_position_circle(self,time):
        ang_rate = 0.15 * np.pi / 2.0
        circ_rad = 2.0
        rd = np.array([circ_rad * np.cos(ang_rate * time), circ_rad * np.sin(ang_rate * time)])
        return rd

    def desired_position_sine(self, time):
        ang_rate = 0.4 * np.pi / 3.0
        circ_rad = 0.2
        x_lim = 10
        x_rate = 0.5
        rd = np.array([time, circ_rad * np.sin(ang_rate * time) + 0.3])
        return rd

    def nonholonomic_kinematics(self, tsim, state, L=0.1):
        x1, x2, x3 = state
        r = np.array([x1,x2])
        if self.rd_type == 'circle':
            rd = self.desired_position_circle(tsim)
        elif self.rd_type == 'sine':
            rd = self.desired_position_sine(tsim)
        else:
            raise NotImplementedError

        theta = x3
        u = student_function_3(rd, r, L, theta)
        v = u.item(0)
        dth = u.item(1)
        dx1 = np.cos(x3) * v
        dx2 = np.sin(x3) * v
        dx3 = dth
        return [dx1, dx2, dx3]

    def integrator(self, times, L):
        x0 = np.array([0, 0, 0])
        dyn = lambda t, x: self.nonholonomic_kinematics(t, x, L=L)
        sol = solve_ivp(dyn, [times[0], times[-1]], x0, t_eval=times)
        x_t = sol.y.T

        if self.rd_type == 'circle':
            x_d_t = np.array(
                [
                    [self.desired_position_circle(time).item(0),self.desired_position_circle(time).item(1)]
                    for time in t
                ]
            )
        elif self.rd_type == 'sine':
            x_d_t = np.array(
                [
                    [self.desired_position_sine(time).item(0),self.desired_position_sine(time).item(1)]
                    for time in t
                ]
            )
        else:
            raise NotImplementedError
        return [x_t, x_d_t]

def plot_result(x_t, x_d_t):
    fig = plt.figure()
    ax = fig.add_subplot(111, autoscale_on=True)
    ax.plot(x_t[:, 0], x_t[:, 1], label=r'$^Ar^R$', c='b')
    ax.plot(x_d_t[:, 0], x_d_t[:, 1], label=r'$^Ar^D$', c='r')
    ax.grid()
    ax.legend()

## Plot the result


In [ ]:
#Follow a sine curve
cls_obj = Simulation('sine')
x_t, x_d_t = cls_obj.integrator(t, L)
plot_result(x_t, x_d_t)

In [ ]:
#Follow a circle curve
cls_obj = Simulation('circle')
x_t, x_d_t = cls_obj.integrator(t, L)
plot_result(x_t, x_d_t)

## Animate Results:

In [ ]:
from matplotlib import animation
from IPython.display import HTML

#Draw the objects
fig3 = plt.figure();
ax3 = fig3.add_subplot(111, autoscale_on=False, xlim=(-10, 10), ylim=(-10, 10));
ax3.set_aspect('equal')

#define animation structions
robot_frame, = ax3.plot([], [], '-',color='green', lw=3)
robot_left_tire, = ax3.plot([], [], '-',color='blue', lw=1)
robot_right_tire, = ax3.plot([], [], '-',color='blue', lw=1)
robot_des_frame, = ax3.plot([], [], 'o',color='red', lw=15)

def update_robot(i, robot_frame, robot_left_tire, robot_right_tire):
    robot_x = x_t[i, 0]
    robot_y = x_t[i, 1]
    theta = x_t[i, 2]

    rx = [-0.2, 0.2]
    ry = [-0.2, 0.2]
    wxl = [-0.215, -0.210]
    wxr = [0.21, 0.215]
    wy = [-0.2, 0.2]

    wl = np.array([[robot_x + np.cos(theta) * wxx, robot_y + np.sin(theta) * wyy] for wxx in wxl for wyy in wy])
    wr = np.array([[robot_x + np.cos(theta) * wxx,robot_y + np.sin(theta) * wyy] for wxx in wxr for wyy in wy])

    r = np.array([[robot_x + np.cos(theta) * rxx, robot_y + np.sin(theta) * ryy] for rxx in rx for ryy in ry])

    # robot frame
    robot_frame_x_ = [-0.2, 0.2, 0.2, -0.2, -0.2]
    robot_frame_y_ = [-0.2, -0.2, 0.2, 0.2, -0.2]
    robot_frame_x = [
        robot_x + np.cos(theta) * robot_frame_x_[j] - np.sin(theta) * robot_frame_y_[j] for j in range(5)
    ]
    robot_frame_y = [
        robot_y + np.sin(theta) * robot_frame_x_[j] + np.cos(theta) * robot_frame_y_[j] for j in range(5)
    ]

    # robot left tire
    robot_left_tire_x_ = [-0.215, 0.210]
    robot_left_tire_y_ = [-0.2, -0.2]
    robot_left_tire_x = [
        robot_x + np.cos(theta) * robot_left_tire_x_[j] - np.sin(theta) * robot_left_tire_y_[j] for j in range(2)
    ]
    robot_left_tire_y = [
        robot_y + np.sin(theta) * robot_left_tire_x_[j] + np.cos(theta) * robot_left_tire_y_[j] for j in range(2)
    ]

    # robot right tire
    robot_right_tire_x_ = [-0.2, 0.2]
    robot_right_tire_y_ = [0.21, 0.21]
    robot_right_tire_x = [
        robot_x + np.cos(theta) * robot_right_tire_x_[j] - np.sin(theta) * robot_right_tire_y_[j] for j in range(2)
    ]
    robot_right_tire_y = [
        robot_y + np.sin(theta) * robot_right_tire_x_[j] + np.cos(theta) * robot_right_tire_y_[j] for j in range(2)
    ]

    # return robot frame
    robot_frame.set_data(robot_frame_x,robot_frame_y)
    robot_left_tire.set_data(robot_left_tire_x, robot_left_tire_y)
    robot_right_tire.set_data(robot_right_tire_x, robot_right_tire_y)
    ax3.set_title(f"Time: {(i * dt):0.2f}")

    # desired point
    p_des_x = x_d_t[i, 0]
    p_des_y = x_d_t[i, 1]
    robot_des_frame.set_data([p_des_x], [p_des_y])

    return robot_frame, robot_left_tire, robot_right_tire, robot_des_frame

robot_ani = animation.FuncAnimation(
    fig3,
    update_robot,
    len(t),
    fargs=(robot_frame, robot_left_tire, robot_right_tire),
    interval=10,
)

robot_ani